In [70]:
class Particle(object):
    def __init__(self,p,m,v):
        if (m == 0):
            self.mass = 1.0
        else:
            self.mass = 2.0
        self.velocity = v
        self.position = p
    
    def __str__(self):
        return format("(%f,%f,%f)" % (self.position, self.mass, self.velocity))
    
    def move(self,time):
        self.position += self.velocity*time
    
    def collide(self,other):
        if (type(other) is Particle or type(other) is Pipe):
            # Particle collision physics - change the velocities of both particles
            # calculate this particle's final velocity
            final_v_1 = ((2*other.mass)/(self.mass + other.mass))*other.velocity
            final_v_1 += ((self.mass - other.mass)/(other.mass + self.mass))*self.velocity
            
            # calculate the other particle's final velocity
            final_v_2 = ((2*self.mass)/(self.mass + other.mass))*self.velocity
            final_v_2 += ((other.mass - self.mass)/(other.mass + self.mass))*other.velocity

            if type(other) is Pipe:
                other.velocity = final_v_2
            else:
                other.velocity = round(final_v_2,1)
            if type(self) is Pipe:
                self.velocity = final_v_1
            else:
                self.velocity = round(final_v_1,1)

The aboce class defines a particle object which we will be using in our model of the gas in a pipe.

The below class defines the pipe itself. 

In [71]:
class Pipe(object):
    def __init__(self, l,m,cp,v):
        self.length = l
        self.mass = m
        self.center = cp
        self.velocity = v
    def move(self,time):
        self.center += self.velocity*time
    def __str__(self):
        return format("(%f,%f,%f,%f)" % (self.length, self.mass, self.center,self.velocity))

We now define our situation. We will create an even number n of particles inside a pipe of length L. These particles will be given random positions to the left half of the pipe's center, random velocities between zero and some maximum, which will define our time step, and random masses of either m1 or m2 where m2 is twice m1.

In [72]:
import random
# The three main needed variables
L = 10.0
half_L = L/2
V_MAX = 10.0
timestep = round(1/V_MAX**2, 2)
N = 100

possible_velocities = []

i = -V_MAX
while i < V_MAX:
    possible_velocities.append(round(i,1))
    i += timestep

# build the particles
particles = []
for i in range (0,N):
    velocity = possible_velocities[random.randrange(0,2*int(V_MAX**2),1)]
    particles.append(Particle(round(random.random()*half_L,1), 
                     random.randint(0,1), 
                     velocity))

# build the pipe 
pipe = Pipe(L, 1000.0,half_L,0.0)

Now that we have our particles and pipe made, we can simulate what will happen. WE will see what happens to each particle, and the pipe, in increments of 1/V_MAX seconds. 

In [73]:
start_time = 0.0
stop_time = 10.0

f = open("pipe_data", "w")

i = 0
while (i < stop_time):
    f.write("%f:\t" % i)
    #for part_p in particles:
    #    f.write("%s\t" % part_p)
    #f.write("\n")
    for part_1 in particles:
        if abs(part_1.position) < 0.01 +(half_L - pipe.center) or part_1.position > (half_L + pipe.center) - 0.01:
            part_1.collide(pipe)
            f.write("Collision %s, wall %s\n" %(part_1, pipe))
        else:
            for part_2 in particles:
                if part_1 != part_2 and abs(part_1.position - part_2.position) < 0.01 and abs(part_1.velocity - part_2.velocity) > 0.01:
                    part_1.collide(part_2)
                    f.write("Collision %s, %s\n" % (part_1, part_2))
                break
    i += timestep
    for part in particles:
        part.move(timestep)
    pipe.move(timestep)